In [ ]:
import torch

In [ ]:
from google.colab import files
uploaded = files.upload()

KeyboardInterrupt: 

In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU is available. Device name:", torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print("GPU not available, using CPU.")


KeyboardInterrupt: 

In [ ]:
!pip install transformers
!pip install accelerate  
!pip install sentencepiece
!pip install pandas

import pandas as pd
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 45.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 81.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 36.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 94.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [ ]:
# 1. Load the model and tokenizer
model_name = "s-nlp/mt0-xl-detox-orpo"

model = AutoModelForSeq2SeqLM.from_pretrained(model_name, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(model_name)

# 2. Define language prompts
LANG_PROMPTS = {
    'zh': '排毒：',
    'es': 'Desintoxicar: ',
    'ru': 'Детоксифицируй: ',
    'ar': 'إزالة السموم: ',
    'hi': 'विषहरण: ',
    'uk': 'Детоксифікуй: ',
    'de': 'Entgiften: ',
    'am': 'መርዝ መርዝ: ',
    'en': 'Detoxify: ',
}

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/834 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/142 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/815 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/16.3M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/416 [00:00<?, ?B/s]

In [ ]:
# 3. Define Detoxification Function
def detoxify(text, lang, model, tokenizer):
    # Concatenate the prompt and the original text
    prompt = LANG_PROMPTS.get(lang, LANG_PROMPTS['en']) + text
    encodings = tokenizer(prompt, return_tensors='pt').to(model.device)

    outputs = model.generate(
        **encodings,
        max_length=128,
        num_beams=10,
        no_repeat_ngram_size=3,
        repetition_penalty=1.2,
        num_beam_groups=5,
        diversity_penalty=2.5,
        num_return_sequences=5,
        early_stopping=True,
    )

    # Return the 1st generated sequence.
    return tokenizer.decode(outputs[0], skip_special_tokens=True)


In [ ]:

input_csv = "hate_speech_data_v2_unique.csv"
df = pd.read_csv(input_csv)

lang_code = 'en'

df['text'] = df['text'].apply(lambda x: detoxify(x, lang_code, model, tokenizer))

output_csv = "hate_speech_data_v2_unique_detoxified.csv"
df.to_csv(output_csv, index=False)

print(f"deotxic CSV were saved in: {output_csv}")

解毒后 CSV 已保存至: hate_speech_data_v2_unique_detoxified.csv


In [ ]:
from google.colab import files
files.download('hate_speech_data_v2_unique_detoxified.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

evluation

In [ ]:
pip install pandas transformers sentencepiece sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 105.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 81.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 34.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 70.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitl

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving hate_speech_data_v2_unique.csv to hate_speech_data_v2_unique.csv
Saving hate_speech_data_v2_unique_detoxified.csv to hate_speech_data_v2_unique_detoxified.csv


In [ ]:
import pandas as pd
import torch

# 1. Load original (toxic) and detoxified data
df_orig = pd.read_csv("hate_speech_data_v2_unique.csv")                 # Original toxic text (text, label=1)
df_detox = pd.read_csv("hate_speech_data_v2_unique_detoxified.csv")     # Detoxified text (text, label=1)

# Assume both CSVs have the same rows in the same order
df = pd.DataFrame({
    "original_text": df_orig["text"],
    "detox_text": df_detox["text"]
})

print("Total rows:", len(df))
print(df.head())

Total rows: 71
                                       original_text  \
0  Typical foreigners behavior. Can I talk to som...   
1  You customer service people are just idiots. I...   
2  I know where your office is. If I don’t get my...   
3  I bet you only hire immigrants because of some...   
4  I know where your office is. If I don’t get my...   

                                          detox_text  
0  Typical foreigners behavior. Can I talk to som...  
1          I'm sick of dealing with people like you.  
2  I know where your office is. If I don’t get my...  
3  I bet you only hire immigrants because of some...  
4  I know where your office is. If I don’t get my...  


In [ ]:
# 2. Load LaBSE and compute semantic similarity
from sentence_transformers import SentenceTransformer, util

labse_model = SentenceTransformer('sentence-transformers/LaBSE')

original_texts = df["original_text"].tolist()
detox_texts = df["detox_text"].tolist()

# Encode texts into embeddings
embeddings_orig = labse_model.encode(original_texts, convert_to_tensor=True)
embeddings_detox = labse_model.encode(detox_texts, convert_to_tensor=True)

# Compute the cosine similarity matrix and extract the diagonal
similarities = util.cos_sim(embeddings_orig, embeddings_detox)
similarity_scores = similarities.diag()

# Store similarity scores in the DataFrame
df["labse_similarity"] = similarity_scores.cpu().numpy()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/461 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/2.02k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/114 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.36M [00:00<?, ?B/s]

In [ ]:
# 3. Load the XLM-R large toxicity detection model
from transformers import pipeline

toxicity_classifier = pipeline(
    task="text-classification",
    model="textdetox/xlmr-large-toxicity-classifier",
    tokenizer="textdetox/xlmr-large-toxicity-classifier"
    # device=0  # Uncomment if you want to use GPU and have one available
)

# 4. Detect toxicity on detoxified texts
results = toxicity_classifier(detox_texts)
# The result items are typically of the form:
# {"label": "toxic"/"neutral"/"hate"/..., "score": 0.99, ...}

labels = [r["label"] for r in results]
scores = [r["score"] for r in results]

df["toxicity_label"] = labels
df["toxicity_score"] = scores

print("\n=== Sample results ===")
print(df.head(10))

# Some models label non-toxic texts as "neutral" rather than "non-toxic".
# Let's unify both into a single "non-toxic" category for correct counting.
df["mapped_toxicity_label"] = df["toxicity_label"].apply(
    lambda x: "non-toxic" if x == "neutral" else x
)
# Statistics: Count how many are labeled "non-toxic"
non_toxic_count = sum(df["mapped_toxicity_label"] == "non-toxic")
detox_success_rate = non_toxic_count / len(df)
print(f"\nProportion labeled as non-toxic: {detox_success_rate:.2%}")

# Calculate average semantic similarity
avg_sim = df["labse_similarity"].mean()
print(f"Average LaBSE similarity: {avg_sim:.3f}")

# Save key columns to CSV
output_filename = "evaluation_result.csv"
df[["original_text", "detox_text", "labse_similarity", "toxicity_score"]].to_csv(output_filename, index=False)

print(f"\n[Done] Created '{output_filename}' with columns: original_text, detox_text, labse_similarity, toxicity_score.")

config.json:   0%|          | 0.00/882 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

Device set to use cuda:0



=== Sample results ===
                                       original_text  \
0  Typical foreigners behavior. Can I talk to som...   
1  You customer service people are just idiots. I...   
2  I know where your office is. If I don’t get my...   
3  I bet you only hire immigrants because of some...   
4  I know where your office is. If I don’t get my...   
5  I refuse to take financial advice from a women...   
6  If you don’t fix this now, I will file a compl...   
7  I know where your office is. If I don’t get my...   
8  I know where your office is. If I don’t get my...   
9  I bet you only hire outsiders because of some ...   

                                          detox_text  labse_similarity  \
0  Typical foreigners behavior. Can I talk to som...          1.000000   
1          I'm sick of dealing with people like you.          0.703145   
2  I know where your office is. If I don’t get my...          1.000000   
3  I bet you only hire immigrants because of some...          0

In [ ]:
from google.colab import files
files.download('evaluation_result.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Andy's

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving detoxified_texts_v2_unique_Andy.csv to detoxified_texts_v2_unique_Andy.csv


In [ ]:
import pandas as pd
import torch


# Load data from CSV
df_raw = pd.read_csv("detoxified_texts_v2_unique_Andy.csv")

# Create a new DataFrame with two columns: "original_text" and "detox_text".
df = pd.DataFrame({
    "original_text": df_raw.iloc[:, 0],
    "detox_text": df_raw.iloc[:, 2]
})

print("Total rows:", len(df))
print(df.head())


# 2. Load LaBSE and compute semantic similarity

from sentence_transformers import SentenceTransformer, util

# Load the multilingual LaBSE model
labse_model = SentenceTransformer('sentence-transformers/LaBSE')

# Encode original and detoxified texts into embeddings
orig_embeddings = labse_model.encode(df["original_text"].tolist(), convert_to_tensor=True)
detox_embeddings = labse_model.encode(df["detox_text"].tolist(), convert_to_tensor=True)

# Compute a cosine similarity matrix
similarities = util.cos_sim(orig_embeddings, detox_embeddings)

# Extract the diagonal since each row in 'orig_embeddings' corresponds
# to the same row in 'detox_embeddings'
similarity_scores = similarities.diag()

# Store the similarity scores in the DataFrame
df["labse_similarity"] = similarity_scores.cpu().numpy()


# 3. Load the XLM-R large toxicity detection model
from transformers import pipeline

toxicity_classifier = pipeline(
    task="text-classification",
    model="textdetox/xlmr-large-toxicity-classifier",
    tokenizer="textdetox/xlmr-large-toxicity-classifier",
    # device=0  # If you have a GPU, you can enable this
)


# 4. Classify detoxified texts for toxicity

results = toxicity_classifier(df["detox_text"].tolist())
# Example output: [{"label": "neutral", "score": 0.999...}, ...]

labels = [res["label"] for res in results]
scores = [res["score"] for res in results]

df["toxicity_label"] = labels
df["toxicity_score"] = scores


# 5. Inspect and compute statistics

print("\n=== Sample results ===")
print(df.head(10))

# If the model outputs 'neutral' as a label for non-toxic texts:
neutral_count = sum(df["toxicity_label"] == "neutral")
neutral_rate = neutral_count / len(df)
print(f"\nProportion labeled as neutral (non-toxic): {neutral_rate:.2%}")

# If you want to map "neutral" to "non-toxic" for consistency:
df["mapped_label"] = df["toxicity_label"].apply(lambda x: "non-toxic" if x == "neutral" else x)
non_toxic_count = sum(df["mapped_label"] == "non-toxic")
non_toxic_rate = non_toxic_count / len(df)
print(f"Proportion labeled as non-toxic (mapped): {non_toxic_rate:.2%}")

# Compute the average LaBSE similarity
avg_similarity = df["labse_similarity"].mean()
print(f"Average LaBSE similarity: {avg_similarity:.3f}")



Total rows: 71
                                       original_text  \
0  Typical foreigners behavior. Can I talk to som...   
1  You customer service people are just idiots. I...   
2  I know where your office is. If I don’t get my...   
3  I bet you only hire immigrants because of some...   
4  I know where your office is. If I don’t get my...   

                                          detox_text  
0               ['can I talk to someone competent?']  
1  ["I've been getting used to dealing with peopl...  
2  ["I know where the office is. If I don't cance...  
3  ["I'd bet that you only hire immigrants based ...  
4                    ['I know where the office is.']  


Device set to use cuda:0



=== Sample results ===
                                       original_text  \
0  Typical foreigners behavior. Can I talk to som...   
1  You customer service people are just idiots. I...   
2  I know where your office is. If I don’t get my...   
3  I bet you only hire immigrants because of some...   
4  I know where your office is. If I don’t get my...   
5  I refuse to take financial advice from a women...   
6  If you don’t fix this now, I will file a compl...   
7  I know where your office is. If I don’t get my...   
8  I know where your office is. If I don’t get my...   
9  I bet you only hire outsiders because of some ...   

                                          detox_text  labse_similarity  \
0               ['can I talk to someone competent?']          0.463382   
1  ["I've been getting used to dealing with peopl...          0.370818   
2  ["I know where the office is. If I don't cance...          0.779589   
3  ["I'd bet that you only hire immigrants based ...          0

In [ ]:
from google.colab import files

df[["original_text", "detox_text", "labse_similarity", "toxicity_score"]].to_csv(
    "Andy_evaluation_result.csv", index=False
)
files.download('Andy_evaluation_result.csv')
print("\n[Done] Saved 'Andy_evaluation_result.csv' with original_text, detox_text, labse_similarity, toxicity_score.")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


[Done] Saved 'Andy_evaluation_result.csv' with original_text, detox_text, labse_similarity, toxicity_score.
